# Evolutionary History of Felidae
---
#### By:  Sean Mathew
---
## Introduction
Felidae is a taxonomic family of cats. Members of Felidae range from the domestic cat to extant big cats, such as lions and tigers, but also extinct members such as the saber-toothed smilodon. This project aims to analyze the diversity of Felidae through space and time. The data utilized in this project is found on the PaleoBiology Database online at paleobiodb.org. Occurrence data of fossils is downloadable as a CSV file with species name, and an estimate for the age of each fossil in the collection. In order to find species diversity through time, the files are cleaned up with UNIX, then run through python functions to get the data into a plottable format, and finally visualized using R. R is used to plot species diversity through time. Since the fossil ages are estimates, we can use PyRate to statistically analyze more accurate estimates for speciation and extinction rates. Throughout the history of Felidae, some interesting events include the Late Miocene Radiation, which is responsible for the great species diversity we see today, and divergenece between modern cats and now extinct saber-toothed cats, such as the smilodon, which spiked another wave of speciation. Diversity graphs will illustrate these events. In addition to occurrence data, the paleobiology database also houses collection data with information on the locations of the fossils as well as their ages. This information can be used to plot the fossil through time, showing the geographic distribution of Felidae through history. Most clades, according to models created by Raia, Carotenuto, et.al, are predicted to fall in species richness after large diversification events like the late Miocene and divergence of sabertooths. Felidae is indeed in decline, and many species, such as the tiger, are endangered today. 

----
### Examples
![Lion](examples_of_felidae/lion.jpg)
![Tiger](examples_of_felidae/tiger.jpg)
![Cat](examples_of_felidae/cat.jpg)
![Sabertooth](examples_of_felidae/sabertooth.jpg)
----

## Functions in python

### get_column_names()

**file must be a formatted csv file with the head removed and any "\r" characters replaced with newlines**

for this particular file, "felidae_occ.csv", I used the following Unix command

tail -n +18 felidae_occ.csv > formatted_felidae_occ.csv

This function takes in a file and returns the names of all its columns with appropriate indeces

In [4]:
def get_column_names(filename):
    in_file = open(filename) #opens the inputted file
    column_names = {} #create an empty dictionary to populate with the column number as the key and the name of the column as the value
    first_line = in_file.readline() #read in the first line, with only the column names
    column_name_keys = first_line.split('","') #take the column names and split them by '","' to create a list
    num_columns = len(column_name_keys) #get the total number of columns
    for column in range(0, num_columns): #make a for loop that iterates from zero to the num_columns
        column_names[column] = column_name_keys[column]
    in_file.close()
    return column_names


get_column_names("formatted_felidae_col.csv")

{0: '"collection_no',
 1: 'record_type',
 2: 'formation',
 3: 'lng',
 4: 'lat',
 5: 'collection_name',
 6: 'collection_subset',
 7: 'collection_aka',
 8: 'n_occs',
 9: 'early_interval',
 10: 'late_interval',
 11: 'max_ma',
 12: 'min_ma',
 13: 'reference_no"\n'}

### column_extractor()

**file must be a formatted csv file with the head removed and any "\r" characters replaced with newlines**

for this particular file, "panthera_col.csv", I used the following Unix command

tail -n +18 panthera_col.csv > formatted_panthera_col.csv


This function will take any two columns and group them together in a dictionary. The keys can be used to identify and match up different values that can be plotted against each other


In [1]:
def column_extractor(file_name, key_column, value_column):
    file_input = open(file_name) #opens the inputted file
    file_input.readline()
    extract_dict = {} #empty dictionary
    for line in file_input:
        y = line.split('","')[key_column] #key column is the y value
        x = line.split('","')[value_column] #value column is the x value
        extract_dict[y] = x #populate dictionary
    file_input.close() 
    return extract_dict




#see section: "Graphing by fossil age" for demonstration of this function

### stratify()
**Must supply a dictionary of items "to stratify" and a "dictionary" to stratify over**

**Both dictionaries must have matching keys**

To supply dictionary items to stratify over (species, time interval, etc.) *For this particular example, I have used time interval in which a fossil shows up as the dictionary to stratify over from the file "formatted_panthera_col.csv"*

This function takes two dictionaries with matching keys as arguments. The first one, "to_stratify" provides as its values all the possible items to stratify over. The other dictionary "dictionary"'s values are stratified over the unique list created from to_stratify. The function returns a list of the values from "dictionary" as dictionaries separated by the to_stratify values. The matching keys are used to match up the values from two different dictionaries.


In [2]:
def stratify(to_stratify, dictionary):   
    stratify_list = list(to_stratify.values())
    stratify = [] #empty list to hold unique values of items to stratify over
    list_of_dicts = [] #empty list to hold stratified dictionaries
    #for loop to create unique list of items to stratify over
    for item in stratify_list:
        if item not in stratify:
            stratify.append(item)
    #for loop to create a dictionary per item in stratify
    for item in stratify:
        new_dict = {}
        #iterate through all items to stratify them into different dictionaries
        for key in to_stratify.keys():
            if to_stratify[key] == item:
                new_dict[key] = dictionary[key] #populate new_dict only if correct dictionary
        list_of_dicts.append(new_dict) #add each item that satisfies the category to the new dictionary
    return list_of_dicts





#see section: "Graphing by fossil age" for demonstration of this function

### map_plotter
**Must supply two dictionaries with matching keys**

The function takes in two dictionaries with matching keys and plots both dictionaries' values as a scatterplot on top of a map, and a filename. The map is created with this filename

In [3]:
def map_plotter(x_dict, y_dict, filename):
    lattitude = [] #empty list for lattitudes
    longitude = [] #empty list for longitudes
    
    #extract the values from x_dict into a list so it is plottable
    x_list = list(x_dict.keys()) #make a list of all the keys
    for item in x_list:
        longitude.append(float(x_dict[item])) #make sure value is a float
    
    #do the same for lattitude
    y_list = list(y_dict.keys()) 
    for item in y_list:
        lattitude.append(float(y_dict[item]))
    
    import matplotlib.pyplot as plt
    import numpy as np
    from mpl_toolkits.basemap import Basemap
    #plot and return
    plt.figure(figsize = (20,10))
    map = Basemap()
    map.drawcoastlines()
    x,y = map(longitude, lattitude)
    map.scatter(x,y,marker="D", color="r")
    plt.savefig(filename)
    return




#see section: "Graphing by fossil age" for demonstration of this function

### extract_fossil_ages()
**Must supply filename of csv file downloaded from paleodb.org, and must be formatted with the header removed using tail function in UNIX**

This function takes in a filename and outputs a dictionary with the each species in the collection as a key and a list of the average fossil age of each occurence belonging to the species as a value 

In [4]:
def extract_fossil_ages(filename):
    from collections import defaultdict
    species_ranges = defaultdict(list) #values of the dictionary is a list
    genera = open(filename) #input file
    genera.readline() #read out first line
    genera_dict= {} #create empty dict to return
    for line in genera: #iterating through one line at a time
        species = line.split('","')[5] #species is sixth item
        min_ma = float(line.split('","')[15]) #minimum estimate of age
        max_ma = float(line.split('","')[14]) #maximum estimate of age
        avg_ma = (max_ma + min_ma )/ 2 #find average estimate of age
        genus = species.split(" ")[0] #genus is first word of species name
        if species != genus: #filters out the genus only entries
            species_ranges[species].append(avg_ma) #populate dictionary
    genera.close()
    return species_ranges



#see section: "Output to R: Fossil Occurrences" for demonstration of this function

### dict_to_file()
**Must supply dictionary produced via extract_fossil_ages()**

This function takes in a dictionary created through the previous function, extract_fossil_ages() and a new filename and returns a file with that filename. The file will take the dictionary supplied and for each species find the maximum and minimum of the list of average ages that are the values to the dictionary. 

In [5]:
def dict_to_file(species_ranges_dict, new_filename):
    output_file = open(new_filename, "w") #output file created
    alphabetical = sorted(list(species_ranges_dict.keys())) #dictionary may not be sorted
    for key in alphabetical: #iterating through sorted keys
        species = key #key is species name
        
        ages = species_ranges_dict[key] #take value, which is a list
        minage = min(ages) #min average fossil age
        maxage = max(ages) #max average fossil age
        genus = key.split(" ")[0] #first world of species name is genus
        output_file.write(genus + "," + species + "," + str(minage) + "," + str(maxage) + "\n") #write to output file
    
    output_file.close()
    return output_file



#see section: "Output to R: Fossil Occurrences" for demonstration of this function

## Graphing by fossil age

In [6]:
min_ma = column_extractor("formatted_felidae_col.csv", 0, 11) #extract lower bound of fossil age
max_ma = column_extractor("formatted_felidae_col.csv", 0, 12) #extract upper bound of fossil age
#iterate through data, find average fossil age, add it to a new dictionary with average fossil age for each fossil
avg_ma = {}
for key in min_ma.keys():
    avg = ( float(max_ma[key]) + float(min_ma[key]) ) / 2
    avg_ma[key] = avg
    
longitude = column_extractor("formatted_felidae_col.csv", 0, 3) #extract longitude column
latitude = column_extractor("formatted_felidae_col.csv", 0, 4) #extract latitude column

strat_lng2 = stratify(avg_ma, longitude) #stratify longitude by avg_ma
strat_lat2 = stratify(avg_ma, latitude) #stratify latitude by avg_ma


strat_lng3 = []
strat_lat3 = []

for i in range(0, len(strat_lng2)):
    new_dict = {}
    index = i
    while(index > 0):
        for key in strat_lng2[index].keys():
            new_dict[key] = strat_lng2[index][key]
        index = index - 1
    strat_lng3.append(new_dict)

    
for i in range(0, len(strat_lat2)):
    new_dict = {}
    index = i
    while(index > 0):
        for key in strat_lat2[index].keys():
            new_dict[key] = strat_lat2[index][key]
        index = index - 1
    strat_lat3.append(new_dict)
#this portion can be better done in r, where each new graph can be added to the last one


#this for loop will go through the fossils by avg_ma

for i in range(0, len(strat_lng3)):    
    x = strat_lng3[i]
    y = strat_lat3[i]
    filename = "maps/map{}.png".format(i)
    map_plotter(x, y, filename)




/home/eeb177-student/miniconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3222: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/home/eeb177-student/miniconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3231: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)
/home/eeb177-student/miniconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


### Converting to GIF

In [10]:
import imageio
    
filenames =[]
for i in range(0, 48):
    filenames.append("maps/map{}.png".format(i))
images = []

for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('maps/movie.gif', images)
    

![](maps/movie.gif)

---
## Output to R: Fossil Occurrences

In [10]:
felid_ranges = extract_fossil_ages("formatted_felidae_occ.csv")

dict_to_file(felid_ranges, "felidae_ranges.csv")


<_io.TextIOWrapper name='felidae_ranges.csv' mode='w' encoding='UTF-8'>

## R: Fossil Occurrences and Species Diversity 


title: "Felidae Analysis"
author: "Sean Mathew"
date: "March 9, 2017"
output: md_document

### Reading in data
```{r}
library(ggplot2)
felids <- read.csv("felidae_ranges.csv", header = F, as.is = T)
names(felids) <- c("genus", "species", "minage", "maxage")
head(felids)
```

### Occurrences
```{r}
library("forcats")
felids <- felids %>% arrange(maxage)
felids$maxage <- felids$maxage+0.5
felid_occ <- ggplot(felids, aes( x = fct_reorder(species, minage, .desc = T), maxage, colour = genus))
felid_occ <- felid_occ + geom_linerange(aes(ymin = minage, ymax = maxage + 0.5))
felid_occ <- felid_occ + theme(legend.position="none")
felid_occ <- felid_occ + coord_flip()
felid_occ <- felid_occ +  theme(axis.text.y = element_text(size=3))
felid_occ <- felid_occ + theme(axis.ticks.y=element_blank())
felid_occ <- felid_occ + scale_y_continuous(limits=c(0, 20), expand = c(0, 0), breaks=c(0, 5, 10, 15, 20))
felid_occ <- felid_occ + labs(title = "Felid Fossil Occurrences", x = "Species", y = "Ma ago") + theme(plot.title = element_text(hjust = 0.5, size=22, face = "bold"), axis.title =element_text(size=20))

felid_occ
```


```{r}
ggsave(filename = "output_pdf/felid_occ.pdf", plot = felid_occ)

```
### Diversity

```{r}
library(tidyr)
library(dplyr)
diversity <- felids %>% gather(key = type, value = age, minage, maxage) %>% mutate(count = ifelse(type == "maxage", 1, -1)) %>% group_by(age) %>% summarise(count = sum(count))  %>% arrange(-age, -count) %>% mutate(diversity = cumsum(count)) 

felid_div <- ggplot(diversity, aes(x = age, y = diversity)) + geom_step()

felid_div <- felid_div + labs(title = "Felidae Diversity Through Time", x = "Ma ago", y = "Number of Species") + theme(plot.title = element_text(hjust = 0.5, size=22, face = "bold"), axis.title =element_text(size=20))
felid_div

```

```{r}
ggsave(filename = "output_pdf/felid_div.pdf", plot = felid_div)

```



## Graphs from R
**Created using ggplot() in RStudio**

![alt](output_png/felid_occ.png)

![alt](output_png/felid_div.png)

## Graphs from PyRate
**run UNIX script "PyRate/full-pyrate-run.sh"** (link below)

![](output_png/pyrate_results.png)



## References
Christiansen, Per. "Evolution of Skull and Mandible Shape in Cats (Carnivora: Felidae)." PLOS (2008): n. pag. PLOS. Web. <http://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0002807&type=printable>. 

Johnson, Warren E., William J. Murphy, Agostinho Antunes, Emma Teeling, Stephen J. O’Brien, Jill Pecon-Slattery, and Eduardo Eizirik. "The Late Miocene Radiat Ion of Modern Felidae : A Genetic Assessment." Science (2006): n. pag. Web. <http://www.imaginiquebengals.com/johnsonsuppl2006.pdf>. 

Raia P., Carotenuto F., Mondanaro A., Castiglione S., Passaro F., Saggese F., Melchionna M., Serio C., Alessio L., Silvestro D., Fortelius M. 2016. Progress to extinction: increased specialisation causes the demise of animal clades. Sci. Rep. 6:30965.<http://www.nature.com/articles/srep30965>.



---

### Github link:
https://github.com/seanmathew95/eeb-177-final-project

---

### PyRate materials: 
https://github.com/seanmathew95/eeb-177-final-project/blob/master/PyRate/full-pyrate-run.sh
https://github.com/seanmathew95/eeb-177-final-project/blob/master/PyRate/process_felid_data.R

---
### Rmarkdown file:
https://github.com/seanmathew95/eeb-177-final-project/blob/master/Analysis_of_Felidae.md
